ESTUDO PARA A PROVA:

REFAZENDO TODOS OS EXERCÍCIOS POSSÍVEIS


# **GPU AULA13**

Para compilar no Insper:

nvcc -arch=sm_70 -std=c++14 exemplo1-criacao-iteracao.cu -o exemplo1


Continuando o exercício anterior, calcule as seguintes medidas. Não se esqueça de passar o device_vector para a sua função reduce

- O preço médio das ações nos últimos 10 anos.

- O preço médio das ações no último ano (365 dias atrás).

- O maior e o menor preço da sequência inteira e do último ano.


In [64]:
%%writefile aula13.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>
 using namespace std;


int main(){
  int n;
  cin >> n;
  thrust::host_vector<double>acoes(n, 0);
  for (int i = 0; i < n; i++){
    cin >> acoes[i]; 
  }

  thrust::device_vector<double> acoes_gpu(acoes);
  double total_sum = thrust::reduce(acoes_gpu.begin(), acoes_gpu.end(), 0, thrust::plus<double>());
  double media = total_sum/(double) n;

  double num_inicial = acoes_gpu[0];

  thrust::maximum<double> maior;
  double maior_preco = thrust::reduce(acoes_gpu.begin(), acoes_gpu.end(), num_inicial, maior);

  thrust::minimum<double> menor;
  double menor_preco = thrust::reduce(acoes_gpu.begin(), acoes_gpu.end(), num_inicial, menor);

  cout << "Média calculada: " << media << endl;
  cout << "Maior preco: " << maior_preco << endl;
  cout << "Menor preco: " << menor_preco << endl;

  return 0;
}

Overwriting aula13.cu


In [65]:
!nvcc aula13.cu -o aula13

In [66]:
!./aula13 < stocks.txt

Média calculada: 291.3
Maior preco: 300.511
Menor preco: 278.901


Vamos agora trabalhar com o arquivo stocks2.csv. Ele contém a série histórica de ações da Apple e da Microsoft. Seu objetivo é calcular a diferença média entre os preços das ações AAPL e MSFT.

Dica: quebre o problema em duas partes. Primeiro calcule a diferença entre os preços e guarde isto em um vetor. Depois compute a média deste vetor.

In [78]:
%%writefile aula13_pt2.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>
 using namespace std;


 struct subtracao_modulo
{
    __host__ __device__
    double operator()(const double& x, const double& y) {
          double diferenca = (x - y);
          if (diferenca < 0.0){
            diferenca = diferenca*(-1.0);
          }
          return diferenca;
    }
};



 int main(){
  int n = 10;

  thrust::host_vector<double>acoes_aapl_cpu(n, 0);
  thrust::host_vector<double>acoes_msft_cpu(n, 0);

  for (int i = 0; i < n; i++){
    cin >> acoes_aapl_cpu[i];
    //cout << "AAPL: " << acoes_aapl_cpu[i] << endl;
    cin >> acoes_msft_cpu[i];
    //cout << "MSFT: " << acoes_msft_cpu[i] << endl;
  }

  thrust::device_vector<double>acoes_aapl_gpu(acoes_aapl_cpu);
  thrust::device_vector<double>acoes_msft_gpu(acoes_msft_cpu);

  thrust::device_vector<double> diferencas(n, 0);

  thrust::transform(acoes_aapl_gpu.begin(), acoes_aapl_gpu.end(), acoes_msft_gpu.begin(), diferencas.begin(), subtracao_modulo());

  double soma_diferencas = thrust::reduce(diferencas.begin(), diferencas.end());
  double media_diferencas = soma_diferencas/(double) n;

  cout << "Soma das diferenças: " << soma_diferencas << endl;
  cout << "Média das diferenças: " << media_diferencas << endl;

  return 0;

 }

Overwriting aula13_pt2.cu


In [79]:
!nvcc aula13_pt2.cu -o aula13_pt2

In [80]:
!./aula13_pt2 < stocks2.csv

Soma das diferenças: 33.6301
Média das diferenças: 3.36301


# **GPU AULA14**



In [81]:
%%writefile aula14_pt1.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>
 using namespace std;


 struct subtracao_modulo
{
    __host__ __device__
    double operator()(const double& x, const double& y) {
          double diferenca = (x - y);
          if (diferenca < 0.0){
            diferenca = diferenca*(-1.0);
          }
          return diferenca;
    }
};



 int main(){
  int n = 10;

  thrust::host_vector<double>acoes_aapl_cpu(n, 0);
  thrust::host_vector<double>acoes_msft_cpu(n, 0);

  for (int i = 0; i < n; i++){
    cin >> acoes_aapl_cpu[i];
    //cout << "AAPL: " << acoes_aapl_cpu[i] << endl;
    cin >> acoes_msft_cpu[i];
    //cout << "MSFT: " << acoes_msft_cpu[i] << endl;
  }

  thrust::device_vector<double>acoes_aapl_gpu(acoes_aapl_cpu);
  thrust::device_vector<double>acoes_msft_gpu(acoes_msft_cpu);

  thrust::device_vector<double> diferencas(n, 0);

  thrust::transform(acoes_aapl_gpu.begin(), acoes_aapl_gpu.end(), acoes_msft_gpu.begin(), diferencas.begin(), subtracao_modulo());

  //PASSO 1 - CALCULO DA MEDIA
  double soma_diferencas = thrust::reduce(diferencas.begin(), diferencas.end());
  double media_diferencas = soma_diferencas/(double) n;

  // Parte 1 da aula 14, a variancia das diferencas:

  thrust::device_vector<double> media_vetor(n, media_diferencas);
  //thrust::fill(media_vetor.begin(), media_vetor.begin()+n, media_diferencas);


  // CALCULO DE (x - mu)
  thrust::transform(diferencas.begin(), diferencas.end(), media_vetor.begin(), diferencas.begin(), thrust::minus<double>());

  // CALCULO DE (x - mu)^2

  thrust::transform(diferencas.begin(), diferencas.end(), diferencas.begin(), thrust::square<double>());

  // PASSO 4  SUM (x - mu)^2/ N
  double variancia = thrust::reduce(diferencas.begin(), diferencas.end())/double(n);
  double std = sqrt(variancia);

  cout << "Soma das diferenças: " << soma_diferencas << endl;
  cout << "Média das diferenças: " << media_diferencas << endl;
  cout << "Variancia " << variancia << endl;
  cout << "Desvio padrão: " << std << endl;

  return 0;

 }

Writing aula14_pt1.cu


In [82]:
!nvcc aula14_pt1.cu -o aula14_pt1

In [84]:
!./aula14_pt1 < stocks2.csv

Soma das diferenças: 33.6301
Média das diferenças: 3.36301
Variancia 0.224582
Desvio padrão: 0.4739


In [100]:
%%writefile aula14_pt2.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>
 using namespace std;

int main(){
  int n;
  cin >> n;
  thrust::host_vector<double>acoes(n, 0);
  for (int i = 0; i < n; i++){
    cin >> acoes[i]; 
  }

  thrust::device_vector<double> acoes_gpu(acoes);
  thrust::device_vector<double> ganho_diario(n-1);

  thrust::transform(acoes_gpu.begin()+1, acoes_gpu.end(), acoes_gpu.begin(), ganho_diario.begin(), thrust::minus<double>());

  cout << "Ganho diário: " << endl;
  for (int i = 0; i < int(n-1); i++){
    cout << ganho_diario[i] << endl;
  }

  double soma_ganho = thrust::reduce(ganho_diario.begin(), ganho_diario.end());
  double media_ganho = soma_ganho/(double) (n-1);

  cout << "Média do ganho diário: " << media_ganho << endl;
}



Overwriting aula14_pt2.cu


In [101]:
!nvcc aula14_pt2.cu -o aula14_pt2

In [102]:
!./aula14_pt2 < stocks.txt

Ganho diário: 
3.2077
7.9914
9.4916
-5.9556
6.875
-4.3139
-3.7179
2.0256
-4.9554
Média do ganho diário: 1.18317


In [114]:
%%writefile aula14_pt3.cu

 #include <thrust/host_vector.h>
 #include <thrust/device_vector.h>
 #include <thrust/generate.h>
 #include <thrust/functional.h>
 #include <thrust/copy.h>
 #include <cstdlib>
 #include <algorithm>
 #include <iostream>
 #include <iomanip>
 using namespace std;

struct greater_than_zero
{
  __host__ __device__
  bool operator()(const double &x)
  {
    return x > 0.0;
  }
};


struct less_than_zero
{
  __host__ __device__
  bool operator()(const double &x)
  {
    return x < 0.0;
  }
};

int main(){
  int n;
  cin >> n;
  thrust::host_vector<double>acoes(n, 0);
  for (int i = 0; i < n; i++){
    cin >> acoes[i]; 
  }

  thrust::device_vector<double> acoes_gpu(acoes);
  thrust::device_vector<double> ganho_diario(n-1);

  thrust::transform(acoes_gpu.begin()+1, acoes_gpu.end(), acoes_gpu.begin(), ganho_diario.begin(), thrust::minus<double>());

  cout << "Ganho diário: " << endl;
  for (int i = 0; i < int(n-1); i++){
    cout << ganho_diario[i] << endl;
  }

  double soma_ganho = thrust::reduce(ganho_diario.begin(), ganho_diario.end());
  double media_ganho = soma_ganho/(double) (n-1);

  // Para contar quantas vezes da 1: int result = thrust::count(vec.begin(), vec.end(), 1);
  // Contar quantas vezes aumentou de fato  

  int qtd_aumentos = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), greater_than_zero());


  // Calcular o aumento médio
  
  thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), less_than_zero(), 0);
  double soma_aumentos = thrust::reduce(ganho_diario.begin(), ganho_diario.end());
  double media_aumentos = soma_aumentos/(double) qtd_aumentos;

  cout << "Média do ganho diário: " << media_ganho << endl;
  cout << "Quantas vezes aumentou " << qtd_aumentos << endl;
  cout << "Média dos aumentos " << media_aumentos << endl;
}



Overwriting aula14_pt3.cu


In [115]:
!nvcc aula14_pt3.cu -o aula14_pt3

In [116]:
!./aula14_pt3 < stocks.txt

Ganho diário: 
3.2077
7.9914
9.4916
-5.9556
6.875
-4.3139
-3.7179
2.0256
-4.9554
Média do ganho diário: 1.18317
Quantas vezes aumentou 5
Média dos aumentos 5.91826


# **GPU AULA15**

A magnitude de um vetor consiste na raiz quadrada da soma do quadrado de seus elementos. Dessa forma, você deve complementar o código abaixo, de modo a criar uma transformação customizada square, a qual faz uma transformação no vetor transformando os seus elementos ao quadrado, e posteriormente você deve fazer uma redução, de modo a obter a magnitude do vetor.

In [ ]:
 %%writefile aula15_pt1.cu

 #include <iostream>
 #include <iomanip>
 #include <cstdlib>
 #include <chrono>
 #include <cstdlib>
 #include <algorithm>
//INSIRA AS IMPORTACOES NECESSARIAS DA THRUST
 #include <cmath>
 #include <random>

 using namespace std::chrono;
 using namespace std;

 void reportTime(const char* msg, steady_clock::duration span) {
     auto ms = duration_cast<milliseconds>(span);
     std::cout << msg << " - levou - " <<
      ms.count() << " milisegundos" << std::endl;
 }

 // CRIE UMA FUNCTOR PARA CALCULAR A SQUARE
struct square
{
    __host__ __device__
    double operator()(const double& x) {
           return x*x;
    }
};


 // IMPLEMENTE O CALCULO DA MAGNITUDE COM THRUST
 float magnitude(thrust::device_vector<double> x) {
     double result;

     thrust::transform(x.begin(), x.end(), x.begin(), square());
     result = sqrt(thrust::reduce(x.begin(), x.end()));

     return result;

     // OUUUUU 
     //thrust::transform_reduce(x.begin(), x.end(), square(), 0.0f, thrust::plus<double>());
 }

 int main(int argc, char** argv) {
     if (argc != 2) {
         std::cerr << argv[0] << ": numero invalido de argumentos\n"; 
         std::cerr << "uso: " << argv[0] << "  tamanho do vetor\n"; 
         return 1;
     }
     int n = std::atoi(argv[1]); //numero de elementos
     steady_clock::time_point ts, te;

     // Faça um  vector em thrust 

     thrust::host_vector<double> v(n);

     // inicilize o  vector
     ts = steady_clock::now();

     std::generate(v.begin(), v.end(), std::rand);

     thrust::device_vector<double> v_d(v);

     te = steady_clock::now();
     reportTime("Inicializacao", te - ts);

     // Calcule a magnitude do vetor
     ts = steady_clock::now();
     float len = magnitude(v_d);
     te = steady_clock::now();
     reportTime("Tempo para calculo", te - ts);


     std::cout << std::fixed << std::setprecision(4);
     std::cout << "Magnitude : " << len << std::endl;
 }

# **AULA 10: INTRODUCAO PARALELISMO**

Após implementar a versão serial e calcular o tempo de sua execução, implemente a versão em openmp. Você deve fazer sua implementação de duas formas:

a) A primeira implementação você deve trabalhar com 2 threads e a partir do id da thread, você deve dividir a soma em duas partes, cada thread executando a sua porção.

In [28]:
%%writefile leibniz_parte1.cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <fstream>
#include <omp.h>
using namespace std;

int main(){
  long int n = 1000000000;
  int multiplier = -1;

  double soma_serie = 0.0;

  omp_set_num_threads(2);


  #pragma omp parallel 
  {
    int thread_id = omp_get_thread_num();
    int num_threads = omp_get_num_threads();

    long int start = (n / num_threads) * thread_id;
    long int end = (n / num_threads) * (thread_id + 1);
    
    if (thread_id == num_threads - 1) {
      end = n;
    }

    double soma_local = 0.0;

    for (int i = start; i < end; i++){
      multiplier *= -1.0;
      soma_local+= (double)multiplier/(double)(2*i + 1.0);
    }

    #pragma omp critical
    soma_serie += soma_local;

  }


  double pi = double(soma_serie)*4.0;
  cout << "Aproximação do pi: " << pi << endl;

  return 0;
}

Overwriting leibniz_parte1.cpp


In [29]:
!g++ -Wall -O3 -fopenmp leibniz_parte1.cpp -o leibniz_parte1


In [30]:
!./leibniz_parte1

Aproximação do pi: 3.14159


b) A segunda implementação você deve trabalhar com for do openmp e tratar como uma redução do valor de PI. Calcule o tempo de execução.

In [31]:
%%writefile leibniz_parte2.cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <fstream>
#include <omp.h>
using namespace std;

int main(){
  long int n = 1000000000;
  int multiplier = -1;

  double soma_serie = 0.0;

  omp_set_num_threads(2);

  #pragma omp parallel for reduction(+ : soma_serie)
  for (int i = 0; i < n; i++){
    multiplier *= -1.0;
    soma_serie+= (double)multiplier/(double)(2*i + 1.0);
  }


  double pi = double(soma_serie)*4.0;
  cout << "Aproximação do pi: " << pi << endl;

  return 0;
}

Writing leibniz_parte2.cpp


In [32]:
!g++ -Wall -O3 -fopenmp leibniz_parte2.cpp -o leibniz_parte2


In [33]:
!./leibniz_parte2

Aproximação do pi: 3.14159


Complete exercicio1.cpp criando duas tarefas. A primeira deverá rodar funcao1 e a segunda funcao2. Salve seus resultados nas variáveis indicadas no código.

In [34]:
%%writefile exercicio1_paralelo.cpp
#include <iostream>
#include <unistd.h>
#include <omp.h>
#include <chrono>

double funcao1() {
    sleep(4);
    return 47;
}

double funcao2() {
    sleep(1);
    return -11.5;
}

int main() {
    auto t_start = std::chrono::high_resolution_clock::now();

    double res_func1, res_func2;

    # pragma omp parallel
    {
        #pragma omp task
        {
            std::cout << "Estou rodando na tarefa " << omp_get_thread_num() << "\n";
            res_func1 = funcao1();
        }

        #pragma omp task
        {
          res_func2 = funcao2();
        }
    }

    //res_func1 = funcao1();
    //res_func2 = funcao2();

    std::cout << res_func1 << " " << res_func2 << "\n";

    auto t_end = std::chrono::high_resolution_clock::now();

    double elapsed_time_ms = std::chrono::duration<double, std::milli>(t_end-t_start).count();
    double elapsed_time_s = elapsed_time_ms/1000;
    std::cout << "Tempo de execução: " << elapsed_time_s << " s" << std::endl;

}

Writing exercicio1_paralelo.cpp


In [35]:
!g++ -g -Wall -fopenmp exercicio1_paralelo.cpp -o exercicio1_paralelo

In [36]:
!./exercicio1_paralelo

Estou rodando na tarefa 0
Estou rodando na tarefa 1
47 -11.5
Tempo de execução: 5.00055 s


In [37]:
%%writefile exercicio1_sequencial.cpp
#include <iostream>
#include <unistd.h>
#include <chrono>

double funcao1() {
    sleep(4);
    return 47;
}

double funcao2() {
    sleep(1);
    return -11.5;
}

int main() {
    auto t_start = std::chrono::high_resolution_clock::now();

    double res_func1, res_func2;

    res_func1 = funcao1();
    res_func2 = funcao2();

    std::cout << res_func1 << " " << res_func2 << "\n";

    auto t_end = std::chrono::high_resolution_clock::now();

    double elapsed_time_ms = std::chrono::duration<double, std::milli>(t_end-t_start).count();
    double elapsed_time_s = elapsed_time_ms/1000;
    std::cout << "Tempo de execução: " << elapsed_time_s << " s" << std::endl;

}

Writing exercicio1_sequencial.cpp


In [38]:
!g++ -g -Wall -fopenmp exercicio1_sequencial.cpp -o exercicio1_sequencial

In [39]:
!./exercicio1_sequencial

47 -11.5
Tempo de execução: 5.00051 s


# **AULA 11: SCHEDULING E EFEITOS COLATERAIS**

In [40]:
%%writefile pi_recursivo_sequencial.cpp
#include <omp.h>
#include <iostream>
#include <iomanip>
static long num_steps = 1024l*1024*1024*2;

#define MIN_BLK  1024*1024*256

double sum = 0;

void pi_r(long Nstart, long Nfinish, double step) {
    long i,iblk;
    if (Nfinish-Nstart < MIN_BLK){
        for (i = Nstart; i < Nfinish; i++){
            double x = (i+0.5)*step;
            sum += 4.0/(1.0+x*x); 
        }
    } else {
        iblk = Nfinish-Nstart;
        pi_r(Nstart,         Nfinish-iblk/2,step);
        pi_r(Nfinish-iblk/2, Nfinish,       step);
    }
}

int main () {
    long i;
    double step, pi;
    double init_time, final_time;
    step = 1.0/(double) num_steps;
    init_time = omp_get_wtime();
    pi_r(0, num_steps, step);
    pi = step * sum;
    final_time = omp_get_wtime() - init_time;

    std::cout << "for " << num_steps << " steps pi = " << std::setprecision(15) << pi << " in " << final_time << " secs\n";
}

Writing pi_recursivo_sequencial.cpp


In [41]:
!g++ -Wall -fopenmp pi_recursivo_sequencial.cpp -o pi_recursivo_sequencial

pi_recursivo_sequencial.cpp: In function ‘int main()’:
pi_recursivo_sequencial.cpp:25:10: warning: unused variable ‘i’ [-Wunused-variable]
   25 |     long i;
      |          ^


In [42]:
!./pi_recursivo_sequencial

for 2147483648 steps pi = 3.14159265358977 in 32.557747029 secs


In [43]:
%%writefile pi_recursivo_parallel.cpp
#include <omp.h>
#include <iostream>
#include <iomanip>
static long num_steps = 1024l*1024*1024*2;

#define MIN_BLK  1024*1024*256

double sum = 0;

void pi_r(long Nstart, long Nfinish, double step) {
    long i,iblk;
    if (Nfinish-Nstart < MIN_BLK){
        #pragma omp parallel for reduction(+ : sum)
        for (i = Nstart; i < Nfinish; i++){
            double x = (i+0.5)*step;
            sum += 4.0/(1.0+x*x); 
        }
    } else {
        iblk = Nfinish-Nstart;
        pi_r(Nstart,         Nfinish-iblk/2,step);
        pi_r(Nfinish-iblk/2, Nfinish,       step);
    }
}

int main () {
    long i;
    double step, pi;
    double init_time, final_time;
    step = 1.0/(double) num_steps;
    init_time = omp_get_wtime();
    pi_r(0, num_steps, step);
    pi = step * sum;
    final_time = omp_get_wtime() - init_time;

    std::cout << "for " << num_steps << " steps pi = " << std::setprecision(15) << pi << " in " << final_time << " secs\n";
}

Writing pi_recursivo_parallel.cpp


In [44]:
!g++ -Wall -fopenmp pi_recursivo_parallel.cpp -o pi_recursivo_parallel

pi_recursivo_parallel.cpp: In function ‘int main()’:
pi_recursivo_parallel.cpp:26:10: warning: unused variable ‘i’ [-Wunused-variable]
   26 |     long i;
      |          ^


In [45]:
!./pi_recursivo_parallel

for 2147483648 steps pi = 3.14159265358977 in 18.0939683519996 secs


In [46]:
%%writefile pi_recursivo_task.cpp

#include <omp.h>
#include <iostream>
#include <iomanip>
static long num_steps = 1024l * 1024 * 1024 * 2;

#define MIN_BLK 1024 * 1024 * 256

double pi_r(long Nstart, long Nfinish, double step) {
    long i, iblk;
    double sum = 0.0;

    if (Nfinish - Nstart < MIN_BLK) {
        for (i = Nstart; i < Nfinish; i++) {
            double x = (i + 0.5) * step;
            sum += 4.0 / (1.0 + x * x);
        }
    } else {
        iblk = Nfinish - Nstart;

        #pragma omp task shared(sum)
        {
            double sum1 = pi_r(Nstart, Nfinish - iblk / 2, step);
            #pragma omp atomic
            sum += sum1;
        }

        #pragma omp task shared(sum)
        {
            double sum2 = pi_r(Nfinish - iblk / 2, Nfinish, step);
            #pragma omp atomic
            sum += sum2;
        }

        #pragma omp taskwait
    }

    return sum;
}

int main() {
    long i;
    double step, pi;
    double init_time, final_time;
    step = 1.0 / (double)num_steps;
    init_time = omp_get_wtime();
    #pragma omp parallel
    #pragma omp single  // garante que uma thread só vai criar as tarefas
    {
        pi = pi_r(0, num_steps, step);
    }
    pi *= step;
    final_time = omp_get_wtime() - init_time;

    std::cout << "for " << num_steps << " steps pi = " << std::setprecision(15) << pi << " in " << final_time << " secs\n";
}


Writing pi_recursivo_task.cpp


In [47]:
!g++ -Wall -fopenmp pi_recursivo_task.cpp -o pi_recursivo_task

pi_recursivo_task.cpp: In function ‘int main()’:
pi_recursivo_task.cpp:42:10: warning: unused variable ‘i’ [-Wunused-variable]
   42 |     long i;
      |          ^


In [48]:
!./pi_recursivo_task

for 2147483648 steps pi = 3.14159265358983 in 18.0078044900001 secs


- pragma omp atomic:

  - Esta diretiva é usada para garantir operações atômicas em uma variável compartilhada entre várias tarefas paralelas.

  - No código fornecido, a variável sum é compartilhada entre as tarefas que realizam as chamadas recursivas.

  - Como várias tarefas estão atualizando o valor de sum simultaneamente, é necessário garantir que as operações de atualização sejam feitas de forma atômica para evitar condições de corrida.

  - A cláusula atomic especifica que a operação de incremento += em sum deve ser executada atomicamente.

- Variáveis locais (sum1 e sum2):

  - As variáveis sum1 e sum2 são usadas para armazenar os resultados parciais das chamadas recursivas.
  - Cada tarefa possui sua própria cópia dessas variáveis, garantindo que não haja conflitos de escrita entre as tarefas.
  - Depois que uma tarefa conclui sua chamada recursiva e obtém o resultado parcial, ela atualiza a variável sum compartilhada usando a cláusula atomic.

- #pragma omp taskwait:

  - Essa diretiva é usada para garantir que a thread atual aguarde a conclusão de todas as tarefas criadas por ela antes de prosseguir.
  - No código fornecido, as chamadas recursivas são feitas como tarefas paralelas, mas queremos garantir que todas as tarefas tenham concluído antes de calcular o valor final de π.
  - A diretiva taskwait faz exatamente isso, aguardando a conclusão de todas as tarefas antes de continuar a execução.

# **AULA 12: EFEITOS COLATERAIS II**


```
sum = 0
De i=0 até N:
  sorteie pontos x,y∈[0,1]
  se x2+y2≤1, sum += 1
devolva 4 * sum / N
```

Faça uma implementação sequencial desse algoritmo. Chama seu programa de pi_montecarlo.cpp. Para fins de debug das próximas versões, mostre o valor de sum na saída de erros. Adote N=100 000.

In [56]:
%%writefile pi_montecarlo.cpp
#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <fstream>
#include <omp.h>
using namespace std;

int main(){
    auto t_start = std::chrono::high_resolution_clock::now();
    double sum = 0;
    int N = 100000;

    std::default_random_engine generator(42);
    std::uniform_real_distribution<double> distribution(0.0, 1.0);

    #pragma omp parallel for reduction(+ : sum)
    for (int i = 0; i < N; i++){
      double x = (double) distribution(generator);
      double y = (double) distribution(generator);
      if ((x*x + y*y) <= 1){
        sum+=1;
      }
    }

    double pi = 4.0 * sum/ (double) N;
    cout <<  "Pi aqui: " << pi << endl;

    auto t_end = std::chrono::high_resolution_clock::now();
    double elapsed_time_ms = std::chrono::duration<double, std::milli>(t_end-t_start).count();
    double elapsed_time_s = elapsed_time_ms/1000;
    std::cout << "Tempo de execução: " << elapsed_time_s << " s" << std::endl;
    return 0;
}

Overwriting pi_montecarlo.cpp


In [57]:
!g++ -Wall -fopenmp pi_montecarlo.cpp -o pi_montecarlo

In [58]:
!./pi_montecarlo

Pi aqui: 3.14276
Tempo de execução: 0.0184794 s


In [59]:
%%writefile pi_montecarlo_melhor.cpp

#include <iostream>
#include <vector>
#include <cmath>
#include <algorithm> 
#include <random>
#include <chrono>
#include <stdlib.h> 
#include <fstream>
#include <omp.h>
using namespace std;

int main(){
    auto t_start = std::chrono::high_resolution_clock::now();
    double sum = 0;
    int N = 100000;
    int num_threads = omp_get_max_threads();

    #pragma omp parallel num_threads(num_threads) reduction(+ : sum)
    {
        int thread_id = omp_get_thread_num();
        std::default_random_engine generator(thread_id);
        std::uniform_real_distribution<double> distribution(0.0, 1.0);

        int num_points_per_thread = N / num_threads;
        int start_index = num_points_per_thread * thread_id;
        int end_index = start_index + num_points_per_thread;

        for (int i = start_index; i < end_index; i++){
            double x = (double) distribution(generator);
            double y = (double) distribution(generator);
            if ((x*x + y*y) <= 1){
                sum += 1;
            }
        }
    }

    double pi = 4.0 * sum / (double) N;
    cout <<  "Pi aqui: " << pi << endl;

    auto t_end = std::chrono::high_resolution_clock::now();
    double elapsed_time_ms = std::chrono::duration<double, std::milli>(t_end-t_start).count();
    double elapsed_time_s = elapsed_time_ms/1000;
    std::cout << "Tempo de execução: " << elapsed_time_s << " s" << std::endl;
    return 0;
}


Writing pi_montecarlo_melhor.cpp


In [60]:
!g++ -Wall -fopenmp pi_montecarlo_melhor.cpp -o pi_montecarlo_melhor

In [61]:
!./pi_montecarlo_melhor

Pi aqui: 3.13672
Tempo de execução: 0.0137051 s


# **AULA 12: MPI**


Utilizando como base o código MPI anterior, construa um programa para obter uma aproximação de PI num cluster. Faça execuções em cluster com 8 e 16 processadores.

In [77]:
%%writefile pi_send_receive.c

#include <mpi.h>
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#define SEED_MPI 35791246

int main(int argc, char* argv[])
{
  long niter = 10000000; // número de iterações para fazer o calculo do pi
  int myid; // o id do processo
  int nodenum; // o número do node
  double x,y;  //x,y valor da coordenada aleatória para sortear e ver se está no círculo 
  double z;    //usado para checar se x^2+y^2<=1
  double pi;    //guarda o valor do pi calculado
  int count = 0;

  MPI_Init(&argc, &argv);                 //Start MPI
  MPI_Comm_rank(MPI_COMM_WORLD, &myid);           //get rank of node's process
  MPI_Comm_size(MPI_COMM_WORLD, &nodenum);

  int recieved[nodenum];  // declarando um array para receber os counts
  long recvniter[nodenum];  // declarando um array para receber o numero de iterações 
  srand(SEED_MPI+myid);                       //Give rand() a seed value. Needs to be different on each node
  
  if(myid != 0)
    {
      for (int i = 0; i < niter; i++){
          x= ((double)rand())/RAND_MAX;           //gets a random x coordinate
          y =((double)rand())/RAND_MAX;           //gets a random y coordinate
          z = sqrt(x*x+y*y);                  //Checks to see if number in inside unit circle
          if (z<=1)
          {
              count++;                //if it is, consider it a valid random point
          }
      }
      for(int i=0; i<nodenum; ++i)
      {
           MPI_Send(&count,1,MPI_INT,0,1,MPI_COMM_WORLD);  // tamanho do que tá mandando para o master, tipo, master, o numero da mensagem e para o mundo
           MPI_Send(&niter,1,MPI_LONG,0,2,MPI_COMM_WORLD);
      }
    }
  else if (myid == 0) // se a rede cair pode devolver -1, então é bom especificar para quando é 0
  {
        
      for(int i=0; i<nodenum; ++i)
      {
          MPI_Recv(&recieved[i],nodenum,MPI_INT,MPI_ANY_SOURCE,1,MPI_COMM_WORLD,MPI_STATUS_IGNORE); // não importa de quem vou receber - MPI_ANY_SOURCE
          MPI_Recv(&recvniter[i],nodenum,MPI_LONG,MPI_ANY_SOURCE,2,MPI_COMM_WORLD,MPI_STATUS_IGNORE);
      }
  }
 
   if (myid == 0)                      //if root process
    {
        int finalcount = 0;
        long finalniter = 0;
        for(int i = 0; i<nodenum; ++i)
        {
            finalcount += recieved[i];
            finalniter += recvniter[i];
        }
 
        pi = ((double)finalcount/(double)finalniter)*4.0;               //p = 4(m/n)
        printf("Pi: %f\n", pi);             //Print the calculated value of pi
 
    }

  MPI_Finalize();
  return 0;
}

Overwriting pi_send_receive.c


In [78]:
!smpicc pi_send_receive.c -o pi_send_receive -lm

In [79]:
!smpirun -np 16 -platform simgrid-template-smpi/cluster_crossbar.xml -hostfile simgrid-template-smpi/cluster_hostfile ./pi_send_receive 10000000 16

[0.000000] [xbt_cfg/INFO] Configuration change: Set 'smpi/privatization' to '1'
[0.000000] [xbt_cfg/INFO] Configuration change: Set 'surf/precision' to '1e-9'
[0.000000] [xbt_cfg/INFO] Configuration change: Set 'network/model' to 'SMPI'
[0.000000] [smpi_kernel/INFO] You did not set the power of the host running the simulation.  The timings will certainly not be accurate.  Use the option "--cfg=smpi/host-speed:<flops>" to set its value.  Check https://simgrid.org/doc/latest/Configuring_SimGrid.html#automatic-benchmarking-of-smpi-code for more information.
Pi: 3.141872


In [64]:
!apt install simgrid

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libsimgrid-dev' instead of 'simgrid'
The following additional packages will be installed:
  libsimgrid3.24 lua5.1 simgrid-java
Suggested packages:
  java-virtual-machine
The following NEW packages will be installed:
  libsimgrid-dev libsimgrid3.24 lua5.1 simgrid-java
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 1,644 kB of archives.
After this operation, 5,797 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsimgrid3.24 amd64 3.24+dfsg-3 [1,328 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 lua5.1 amd64 5.1.5-8.1build4 [94.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 simgrid-java amd64 3.24+dfsg-3 [76.5 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal/universe amd64 libsimgrid-dev amd64 3.24+dfsg-3 [145 kB]
Fetched 1,644 kB in 1s (2,018 kB/s)
Selecting

In [65]:
!git clone https://framagit.org/simgrid/simgrid-template-smpi.git

Cloning into 'simgrid-template-smpi'...
remote: Enumerating objects: 482, done.
remote: Counting objects: 100% (482/482), done.
remote: Compressing objects: 100% (338/338), done.
remote: Total 482 (delta 181), reused 434 (delta 139), pack-reused 0
Receiving objects: 100% (482/482), 967.28 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (181/181), done.


In [ ]:
%%writefile pi_send_receive_hibrido.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#include <math.h>
#include <omp.h>
#define SEED_MPI 35791246
 
int main(int argc, char* argv[])
{
    long niter = 10000000;
    int myid;                       //holds process's rank id
    double x,y;                     //x,y value for the random coordinate
    int i, count=0;                 //Count holds all the number of how many good coordinates
    int global_sum_count;
    int global_sum_niter;
    double z;                       //Used to check if x^2+y^2<=1
    double pi;                      //holds approx value of pi
    int reducedcount;
    int reducedniter;
    int nodenum;
 
    MPI_Init(&argc, &argv);                 //Start MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);           //get rank of node's process
    MPI_Comm_size(MPI_COMM_WORLD, &nodenum);
    srand(SEED_MPI+myid);                       //Give rand() a seed value. Needs to be different on each node
 
    omp_set_num_threads(16);

    #pragma omp parallel for reduction(+ : count)   // ou faz assim
    //#pragma omp parallel for shared(count)   // ou faz isso
    for (i=0; i<niter; ++i)                  //main loop
    {
        x= ((double)rand())/RAND_MAX;           //gets a random x coordinate
        y =((double)rand())/RAND_MAX;           //gets a random y coordinate
        z = sqrt(x*x+y*y);                  //Checks to see if number in inside unit circle
        if (z<=1)
        //#pragma omp critical
        {
            count++;                //if it is, consider it a valid random point
        }
    }

    MPI_Reduce(&count, &reducedcount, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);  // Perform reduction on count
    MPI_Reduce(&niter, &reducedniter, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);

    if (myid == 0)                      //if root process
    {
        pi = ((double)reducedcount / (double)(reducedniter)) * 4.0;    //p = 4(m/n)
        printf("Pi: %f\n", pi);             //Print the calculated value of pi
    }
    
 
    MPI_Finalize();                     //Close the MPI instance
    return 0;
}

In [113]:
%%writefile variancia.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#include <math.h>
#include <omp.h>
#define SEED_MPI 35791246

 
int main(int argc, char* argv[])
{
    int myid;                       //holds process's rank id
    int nodenum;
    int n_elementos = atoi(argv[1]);
 
    MPI_Init(&argc, &argv);                 //Start MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &myid);           //get rank of node's process
    MPI_Comm_size(MPI_COMM_WORLD, &nodenum);
 


    srand(SEED_MPI + myid);                  // Define a semente do gerador de números aleatórios
    double* numeros = (double*)malloc(n_elementos * sizeof(double));
    for (int i = 0; i < n_elementos; i++){
      numeros[i] = ((double)rand())/RAND_MAX + (double)(i+1);
      printf("sorteado: %f\n", numeros[i]);
    }

    double soma = 0.0;
    #pragma omp parallel for reduction(+:soma)
    for (int i = 0; i < n_elementos; i++) {
        soma += numeros[i];
    }
    
    double media_local  = soma / (double)n_elementos;

    double soma_quadrados_diferencas = 0.0;
    #pragma omp parallel for reduction(+:soma_quadrados_diferencas)
    for (int i = 0; i < n_elementos; i++) {
        double diferenca = numeros[i] - media_local;
        soma_quadrados_diferencas += diferenca * diferenca;
    }
    
    double variancia_local = soma_quadrados_diferencas / (double) n_elementos;

    double media_global;
    double variancia_global;
    MPI_Reduce(&media_local, &media_global, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&variancia_local, &variancia_global, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    if (myid == 0) {
        media_global /= nodenum;
        variancia_global /= nodenum;
        printf("Media: %f\n", media_global);
        printf("Variancia: %f\n", variancia_global);
    }

    MPI_Finalize();    // Finaliza o MPI
    free(numeros);     // Libera a memória alocada
 

    return 0;
}

Overwriting variancia.c


In [114]:
!smpicc -fopenmp variancia.c -o variancia -lm

In [115]:
!smpirun -np 16 -platform simgrid-template-smpi/cluster_crossbar.xml -hostfile simgrid-template-smpi/cluster_hostfile ./variancia 5

[0.000000] [xbt_cfg/INFO] Configuration change: Set 'smpi/privatization' to '1'
[0.000000] [xbt_cfg/INFO] Configuration change: Set 'surf/precision' to '1e-9'
[0.000000] [xbt_cfg/INFO] Configuration change: Set 'network/model' to 'SMPI'
[0.000000] [smpi_kernel/INFO] You did not set the power of the host running the simulation.  The timings will certainly not be accurate.  Use the option "--cfg=smpi/host-speed:<flops>" to set its value.  Check https://simgrid.org/doc/latest/Configuring_SimGrid.html#automatic-benchmarking-of-smpi-code for more information.
sorteado: 1.399284
sorteado: 2.771741
sorteado: 3.910703
sorteado: 4.756061
sorteado: 5.564359
sorteado: 1.756685
sorteado: 2.183616
sorteado: 3.210157
sorteado: 4.574820
sorteado: 5.997117
sorteado: 1.616103
sorteado: 2.097481
sorteado: 3.015529
sorteado: 4.396333
sorteado: 5.431977
sorteado: 1.475024
sorteado: 2.512396
sorteado: 3.817696
sorteado: 4.716884
sorteado: 5.368256
sorteado: 1.829372
sorteado: 2.421276
sorteado: 3.615774
so